# Assigment 1

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Find stationary equilibrium](#toc2_)    
- 3. [Vary productivity](#toc3_)    
- 4. [Transition path](#toc4_)    
  - 4.1. [Prepare](#toc4_1_)    
  - 4.2. [Find transition path](#toc4_2_)    
  - 4.3. [Analyze](#toc4_3_)    
- 5. [Extra: Optimal policy](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from HANCModel import HANCModelClass

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = HANCModelClass(name='baseline')

In [3]:
par = model.par
ss = model.ss

## 2. <a id='toc2_'></a>[Find stationary equilibrium](#toc0_)

Test household problem:

In [4]:
model.ss.w0 = 1.0
model.ss.w1 = 1.0
model.ss.rK = 0.01
model.ss.phi0 = 1.0
model.ss.phi1 = 2.0

#model.solve_hh_ss(do_print=True)
#model.simulate_hh_ss(do_print=True)

In [5]:
model.solve_hh_ss(do_print=True)
model.simulate_hh_ss(do_print=True)

In [ ]:
for i_beta,beta in enumerate(par.beta_grid[:len(par.beta_grid)//2]):
    
    fig = plt.figure(figsize=(12,4),dpi=100)

    I = par.a_grid < 500

    # a. consumption
    ax = fig.add_subplot(1,2,1)
    ax.set_title(f'consumption ($\\beta = {beta:.4f}$)')

    for i_z,z in enumerate(par.z_grid):
        if i_z%3 == 0 or i_z == par.Nz-1:
            ax.plot(par.a_grid[I],ss.c[i_beta,i_z,I],label=f'z = {z:.2f}')

    ax.legend(frameon=True)
    ax.set_xlabel('savings, $a_{t-1}$')
    ax.set_ylabel('consumption, $c_t$')
    ax.set_xlim(0,100)
    ax.set_ylim(0,5)

    # b. saving
    ax = fig.add_subplot(1,2,2)
    ax.set_title(f'saving ($\\beta = {beta:.4f}$)')

    for i_z,z in enumerate(par.z_grid):
        if i_z%3 == 0 or i_z == par.Nz-1:
            ax.plot(par.a_grid[I],ss.a[i_beta,i_z,I]-par.a_grid[I],label=f'z = {z:.2f}')

    ax.set_xlabel('savings, $a_{t-1}$')
    ax.set_ylabel('savings change, $a_{t}-a_{t-1}$')
    ax.set_xlim(0,100)
    ax.set_ylim(-3,1)

    fig.tight_layout()
    #fig.savefig(f'figs/c_func_{i_beta}.pdf')

Find stationary equilibrium:

In [ ]:
model.find_ss(method='direct',do_print=True)

### step 1: broad search ###

guess ss.K = 1.0000
implied ss.rK = 0.2777
implied ss.w0 = 0.3703
implied ss.w1 = 0.3703
household problem in ss solved in 0.4 secs [447 iterations]
household problem in ss simulated in 0.0 secs [413 iterations]
implied ss.A_hh = 186.1454
clearing_A = -185.14536146

guess ss.K = 2.0000
implied ss.rK = 0.1782
implied ss.w0 = 0.4752
implied ss.w1 = 0.4752
household problem in ss solved in 1.2 secs [1371 iterations]
household problem in ss simulated in 0.1 secs [518 iterations]
implied ss.A_hh = 238.6204
clearing_A = -236.62035171

guess ss.K = 3.0000
implied ss.rK = 0.1375
implied ss.w0 = 0.5499
implied ss.w1 = 0.5499
household problem in ss solved in 33.4 secs [40197 iterations]
household problem in ss simulated in 0.1 secs [592 iterations]
implied ss.A_hh = 275.9644
clearing_A = -272.96441191

guess ss.K = 4.0000
implied ss.rK = 0.1144
implied ss.w0 = 0.6099
implied ss.w1 = 0.6099
household problem in ss solved in 3.4 secs [4130 iterations]
household probl

In [ ]:
for varname in model.varlist:
    print(f'{varname:15s}: {ss.__dict__[varname]:.4f}')

Gamma          : 1.0000
K              : 4.0987
L0             : 0.6667
L1             : 0.6667
rK             : 0.1126
w0             : 0.6153
w1             : 0.6153
Y              : 1.2819
phi0           : 1.0000
phi1           : 2.0000
A_hh           : 4.0987
C_hh           : 1.0000
L_hh           : 1.5000
A              : 4.0987
L0_hh          : nan
L1_hh          : nan
L              : 1.3333
I              : 0.4099
clearing_A     : -0.0000
clearing_L0    : nan
clearing_L1    : nan
clearing_L     : -0.1667
clearing_Y     : -0.1280


In [ ]:
model.info()

settings:
 par.py_hh = True
 par.py_block = True
 par.full_z_trans = False
 par.T = 500

households:
 grids_hh: [a]
 pols_hh: [a]
 inputs_hh: [rK,w0,w1,phi0,phi1]
 inputs_hh_z: []
 outputs_hh: [a,c,l]
 intertemps_hh: [vbeg_a]

aggregate:
 shocks: [phi1]
 unknowns: [K]
 targets: [clearing_A]

blocks (inputs -> outputs):
 production_firm: [K] -> [Gamma,L0,L1,rK,w0,w1,Y]
 hh: [phi1,rK,w0,w1] -> [phi0,A_hh,C_hh,L_hh]
 market_clearing: [A_hh,L0,L1,L_hh,Y,C_hh,K] -> [A,L0_hh,L1_hh,L,I,clearing_A,clearing_L0,clearing_L1,clearing_L,clearing_Y]


In [ ]:
fig = plt.figure(figsize=(12,4),dpi=100)

# b. assets
ax = fig.add_subplot(1,2,2)
ax.set_title('savings')
for i_beta in range(par.Nfix):
    ax.plot(np.insert(par.a_grid,0,par.a_grid[0]),np.insert(np.cumsum(np.sum(ss.D[i_beta],axis=0)),0,0.0)*par.Nfix,label=f'$\\beta = {par.beta_grid[i_beta]:.4f}, \\chi = {par.eta1_grid[i_beta]:.0f}$')
ax.set_xlabel('assets, $a_{t}$')
ax.set_ylabel('CDF')
ax.legend()
fig.tight_layout()

## 3. <a id='toc3_'></a>[Varying productivity](#toc0_)

## 4. <a id='toc4_'></a>[Transition path](#toc0_)

In [ ]:
model.info(only_blocks=True)
model.draw_DAG(figsize=(10,10))

### 4.1. <a id='toc4_1_'></a>[Prepare](#toc0_)

In [ ]:
model.test_hh_path()

In [ ]:
model.test_path()

**Jacobians:**

In [ ]:
model.compute_jacs()

**Higher bond price:** Always less savings due to lower return.

**Higher tax rate:**

1. Lower savings when in effect due to lower income
2. Higher savings when expected due to lower *future* income

In [ ]:
fig = plt.figure(figsize=(12,len(model.inputs_hh_all)*4),dpi=100)

i = 0
for varname in model.inputs_hh:
    
    # a. savings
    i += 1    
    ax = fig.add_subplot(len(model.inputs_hh_all),2,i)
    ax.set_title(f'savings, $a_t$, to {varname}')
    for s in [0,50,150,250]:
        jac_hh_var = model.jac_hh[('A_hh',varname)]
        ax.plot(np.arange(par.T),jac_hh_var[:,s],label=f'shock at {s}')
    
    if i == 1:
        ax.legend(frameon=True,bbox_to_anchor=(2.75,0.0))

    # b. consumption
    i += 1    
    ax = fig.add_subplot(len(model.inputs_hh_all),2,i)
    ax.set_title(f'consumption, $c_t$, to {varname}')
    for s in [0,50,150,250]:
        jac_hh_var = model.jac_hh[('C_hh',varname)]
        ax.plot(np.arange(par.T),jac_hh_var[:,s],label=f'shock at {s}')

### 4.2. <a id='toc4_2_'></a>[Find transition path](#toc0_)

In [ ]:
jump = ss.G*0.01
rho = 0.95
dG = jump*rho**np.arange(par.T)

In [ ]:
model.find_transition_path(shocks={'dG':dG},do_print=True) #important you can only call a variable defined as shock already
path = model.path

### 4.3. <a id='toc4_3_'></a>[Analyze](#toc0_)

The government issues more bonds. Bond price fall and tax increases. Average utility falls.

Expected discounted utility falls.

Initially savings increase both due to taxes (less future income) and higher return (lower bond price). Lower taxes reduce savings.

Consumption inequality decreases. But small effect relative to the lower mean.

**Questions:** Will a utiliarian planner always set the tax rate so it maximizes steady state social welfare?

If the initial tax rate is low, the transition is beneficial. Social welfare increases.

If the initial tax rate is high, the transition is not beneficial. Social welfare increase.

**Insight:** Optimal policy needs to account for the transition.

**Questions:** Could there ne another transition, which makes the policy beneficial?
